In [ ]:
import numpy as np
import time
import sys
import cv2
import matplotlib.pyplot as plt
import networkx as nx

# 이전에 작성한 함수들
from solution import *
from colony import *
from solve import *
from main import *

def parameter_performance(coordinates, NUM_ROWS, NUM_COLS, selected_coordinates, num_sales, alpha_values, beta_values, limit_values, opt2_values):
    results = []
    for alpha in alpha_values:
        for beta in beta_values:
            for limit in limit_values:
                for opt2 in opt2_values:
                    start1 = time.time()
                    G, pos = create_graph(coordinates, selected_coordinates, NUM_ROWS)
                    solver = Solver()
                    colony = Colony(1, 3)
                    ans = solver.solve(G, colony, num_sales, start=1, limit=limit, opt2=opt2, alpha=alpha, beta=beta)
                    distances = []
                    for i in range(len(ans)):
                        solution = ans[i]
                        optimized_path = optimize_route(solution.path, coordinates, NUM_ROWS)
                        distance = calculate_distance(optimized_path, NUM_ROWS)
                        distances.append(distance)
                    mean_distance = np.mean(distances)
                    end1 = time.time()
                    total_time = end1 - start1
                    results.append({
                        'alpha': alpha,
                        'beta': beta,
                        'limit': limit,
                        'opt2': opt2,
                        'mean_distance': mean_distance,
                        'total_time': total_time
                    })
    return results

# 파라미터 범위 정의
alpha_values = [0.1, 0.5, 1.0]
beta_values = [1.0, 1.5, 2.0]
limit_values = [50, 100, 200]
opt2_values = [0, 1, 2]

# 초기 설정
NUM_ROWS = 103
NUM_COLS = 90
num_sales = 2

# 초기 좌표 입력
df = pd.read_excel(filename, engine='openpyxl')
Crime_grid = df.Grid.values.tolist()
selected_coordinates = [num2coord[num] for num in Crime_grid]  
coordinates = generate_coordinates(NUM_ROWS,NUM_COLS)
start_num=num2coord[int(input('출발 그리드 입력: '))]
starting_coordinate_input_y, starting_coordinate_input_x=start_num
starting_coordinate = C2N(int(starting_coordinate_input_x), int(starting_coordinate_input_y), NUM_COLS) #ok
selected_coordinates = [[starting_coordinate_input_y, starting_coordinate_input_x]] + selected_coordinates

# 선택된 좌표로 변환
to_nodes = np.array([C2N(row, col, NUM_COLS) for (row, col) in selected_coordinates])

# 파라미터 별 성능 측정
performance_results = parameter_performance(coordinates, NUM_ROWS, NUM_COLS, selected_coordinates, num_sales, alpha_values, beta_values, limit_values, opt2_values)

# 결과 출력
for result in performance_results:
    print(f"Alpha: {result['alpha']}, Beta: {result['beta']}, Limit: {result['limit']}, Opt2: {result['opt2']}, Mean Distance: {result['mean_distance']}, Total Time: {result['total_time']}")